In [1]:
# https://colab.research.google.com/drive/1ZZXnCjFEOkp_KdNcNabd14yok0BAIuwS#forceEdit=true&sandboxMode=true&scrollTo=wdPxFvHdTLRK

import tensorflow as tf

from tensorflow import keras
from keras import datasets, layers, models, optimizers 
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import pandas as pd
from PIL import Image
from log_training import TrainingLogger

from kerastuner.tuners import RandomSearch
import shutil
import gc

C:\Users\oswal\AppData\Local\Temp\ipykernel_18776\1951207503.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
#  LOAD AND SPLIT DATASET
path_train = r'C:\Users\oswal\Desktop\ImageClassification\elpv-dataset\train_with_bad_images\**\*'
path_test = r'C:\Users\oswal\Desktop\ImageClassification\elpv-dataset\test_images\**\*'

# Prepare image data 
def img_to_np(path, resize = True, extract_labels=False):  
    img_array = []
    labels = []
    fpaths = glob.glob(path, recursive=True)
    for fname in fpaths:
        if(extract_labels): 
            if '_bad' in os.path.basename(fname):
                labels.append(1)  # 1 for outlier
            else:
                labels.append(0)  # 0 for non-outlier
        img = Image.open(fname).convert("L") # Grayscale when using "RGB" you have to change the encoder and decoder 
        if(resize): img = img.resize((64,64))
        img_array.append(np.asarray(img))
    images = np.array(img_array)
    if(extract_labels): return images, np.array(labels)
    return images

test_images, test_labels = img_to_np(path_test, extract_labels=True)
train_images, train_labels = img_to_np(path_train, extract_labels=True)

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [6]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv_1_filters', min_value=32, max_value=128, step=32), (3, 3), 
                            activation='relu', input_shape=(64, 64, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(layers.Conv2D(hp.Int(f'conv_{i+2}_filters', min_value=32, max_value=128, step=32), (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(layers.Dense(2))

    model.compile(optimizer=optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])
    
    return model

tuner = RandomSearch(build_model, 
                     objective='val_accuracy', 
                     max_trials=5, 
                     executions_per_trial=1, 
                     directory='my_dir', 
                     project_name='cnn_tuning')

tuner.search(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


# Instantiate the logger
logger = TrainingLogger(epochs=10, learning_rate=0.001, batch_size=32, optimizer='adam')
# Log each trial data
for trial in tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials)):
    logger.log_trial(trial)

best_model = tuner.get_best_models(num_models=1)[0]

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.663690447807312

Best val_accuracy So Far: 0.663690447807312
Total elapsed time: 00h 01m 00s


In [7]:
# Create and save trial log
for trial in tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials)):
    logger.log_trial(trial)
logger.print_and_save_log()


"epochs": 10,
"learning_rate": 0.001,
"batch_size": 32,
"optimizer": adam,
"train_accuracy": None,
"train_val_accuracy": None,
"train_loss": None,
"train_val_loss": None,
"test_accuracy": None,
"test_loss": None,
"timestamp": 2024-08-28 17:05:40, 
"model_summary": None,
"trials_data": 
{'trial_id': '4', 'hyperparameters': {'conv_1_filters': 32, 'num_conv_layers': 2, 'conv_2_filters': 64, 'dense_units': 192, 'learning_rate': 0.001, 'conv_3_filters': 64, 'conv_4_filters': 64}, 'score': 0.663690447807312, 'best_step': 8} 
{'trial_id': '3', 'hyperparameters': {'conv_1_filters': 128, 'num_conv_layers': 1, 'conv_2_filters': 96, 'dense_units': 192, 'learning_rate': 0.001, 'conv_3_filters': 128, 'conv_4_filters': 64}, 'score': 0.6607142686843872, 'best_step': 9} 
{'trial_id': '2', 'hyperparameters': {'conv_1_filters': 32, 'num_conv_layers': 3, 'conv_2_filters': 64, 'dense_units': 256, 'learning_rate': 0.01, 'conv_3_filters': 64, 'conv_4_filters': 32}, 'score': 0.5178571343421936, 'best_step':

In [8]:
# Optionally, delete the tuning directory to save space
shutil.rmtree('my_dir')